<a href="https://colab.research.google.com/github/alvlagus/Introduction_to_NLP_GB/blob/Lesson_7/gusev_NLP_HW_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 align='center'>Введение в обработку естественного языка</h1>  

<h2 align='center'>Урок 2. Создание признакового пространства</h2>  

<h3 align='left'>Практическое задание:</h3>  

Берем отызывы за лето (из архива с материалами или предыдущего занятия)
1. Учим conv сеть для классификации
2. Рассмотреть 2-а варианта сеточек  

    2.1 Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/
    
    2.2 Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)
 				
Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше

###Загружаем библиотеки:

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

###Загружаем данные:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_excel('/content/drive/MyDrive/Colab_Notebooks/25_NLP/work_7/data/отзывы за лето.xls')
df.sample(5)

,Rating,Content,Date
3855,5,Очень удобно и практично. Я доволен.,2017-08-08
15726,5,очен хороший банк,2017-07-21
17094,2,Перестал работать на Meizu M3 после штатного о...,2017-07-14
788,5,Отличное приложение,2017-08-12
2459,5,Очень удобное приложение.,2017-08-10


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('Rating', axis=1), df['Rating'], test_size=0.2, random_state=42, shuffle=True)
X_train.head()

,Content,Date
2680,Очень удобно!!!,2017-08-10
844,Не приходит смс код для входа в приложение,2017-08-12
18714,Пока все удобно и понятно.,2017-07-01
13842,Удобное приложение. Работает отлично.,2017-07-24
11974,Очень удобное приложение!!,2017-07-28


In [ ]:
y_train.value_counts()

5    11654
1     1822
4     1735
3      724
2      592
Name: Rating, dtype: int64

In [ ]:
disbalance = (y_train.value_counts().sort_index().max()/y_train.value_counts().sort_index()).values
disbalance

array([ 6.39626784, 19.68581081, 16.09668508,  6.71700288,  1.        ])

### Предобработка

In [ ]:
!pip install -qq stop_words
!pip install -qq pymorphy2

In [ ]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in nltk.word_tokenize(txt) if word not in sw]
    return " ".join(txt)

X_train['Content_preprocess'] = X_train['Content'].apply(preprocess_text)
X_test['Content_preprocess'] = X_test['Content'].apply(preprocess_text)
X_train.head()

,Content,Date,Content_preprocess
2680,Очень удобно!!!,2017-08-10,удобно
844,Не приходит смс код для входа в приложение,2017-08-12,приходить смс код вход приложение
18714,Пока все удобно и понятно.,2017-07-01,удобно понятно
13842,Удобное приложение. Работает отлично.,2017-07-24,удобный приложение работать отлично
11974,Очень удобное приложение!!,2017-07-28,удобный приложение


In [ ]:
from collections import Counter

In [ ]:
corpus = (" ".join(X_train['Content_preprocess'])).split()
corpus_count = Counter(corpus)
corpus_count.most_common(10)

[('приложение', 4936),
 ('удобно', 2606),
 ('работать', 1550),
 ('удобный', 1413),
 ('отлично', 1034),
 ('нравиться', 899),
 ('отличный', 821),
 ('хороший', 809),
 ('телефон', 754),
 ('супер', 660)]

In [ ]:
max_words_true = len(corpus_count)
max_words_true

11524

In [ ]:
MAX_WORDS = max_words_true
MAX_WORDS

11524

In [ ]:
vocabulary = {j:i for i, (j, _) in enumerate(corpus_count.most_common()[:MAX_WORDS], 1)}
list(vocabulary.items())[:10]

[('приложение', 1),
 ('удобно', 2),
 ('работать', 3),
 ('удобный', 4),
 ('отлично', 5),
 ('нравиться', 6),
 ('отличный', 7),
 ('хороший', 8),
 ('телефон', 9),
 ('супер', 10)]

In [ ]:
max_len_true = max([len(txt) for txt in X_train['Content_preprocess']])
max_len_true

907

In [ ]:
np.mean([len(txt) for txt in X_train['Content_preprocess']])

41.75010588733588

In [ ]:
MAX_LEN = 100
MAX_LEN

100

In [ ]:
X_train['Content_sequence'] = X_train['Content_preprocess'].apply(lambda text: [vocabulary.get(word, 0) for word in text.split()][:MAX_LEN]+[0]*(MAX_LEN-len(text.split())))
X_test['Content_sequence'] = X_test['Content_preprocess'].apply(lambda text: [vocabulary.get(word, 0) for word in text.split()][:MAX_LEN]+[0]*(MAX_LEN-len(text.split())))
X_train.head()

,Content,Date,Content_preprocess,Content_sequence
2680,Очень удобно!!!,2017-08-10,удобно,"[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
844,Не приходит смс код для входа в приложение,2017-08-12,приходить смс код вход приложение,"[57, 45, 42, 18, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
18714,Пока все удобно и понятно.,2017-07-01,удобно понятно,"[2, 46, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
13842,Удобное приложение. Работает отлично.,2017-07-24,удобный приложение работать отлично,"[4, 1, 3, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
11974,Очень удобное приложение!!,2017-07-28,удобный приложение,"[4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
X_train_array = np.array([[j for j in i] for i in X_train['Content_sequence']])
X_test_array = np.array([[j for j in i] for i in X_test['Content_sequence']])
X_train_array[0][:10]

array([2, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
y_train_array = y_train-1
y_test_array = y_test-1
y_train_array.head()

2680     4
844      1
18714    3
13842    4
11974    4
Name: Rating, dtype: int64

### Модель

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalMaxPooling1D, Dropout, BatchNormalization, Conv1D

In [ ]:
embedding_dim = 300
hidden_dim = 512

In [ ]:
model = Sequential([
    Embedding(MAX_WORDS+1, embedding_dim),
    Conv1D(embedding_dim*2, 3),
    GlobalMaxPooling1D(),
    Dense(hidden_dim, activation='relu'),
    Dense(hidden_dim//2, activation='relu'),

    Dense(units=5, activation='sigmoid')
])

In [ ]:
model.summary()
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              loss_weights=disbalance,
              optimizer='adam',
              metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 300)         3457500   
                                                                 
 conv1d (Conv1D)             (None, None, 600)         540600    
                                                                 
 global_max_pooling1d (Globa  (None, 600)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 512)               307712    
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 5)                 1285      
                                                        

In [ ]:
with tf.device("GPU:0"):
    hh = model.fit(X_train_array, y_train_array, validation_data=(X_test_array, y_test_array), epochs=10)

Epoch 1/10
517/517 [==============================] - 65s 103ms/step - loss: 6.9348 - accuracy: 0.7620 - val_loss: 6.4315 - val_accuracy: 0.7776
Epoch 2/10
517/517 [==============================] - 9s 18ms/step - loss: 5.4352 - accuracy: 0.8053 - val_loss: 6.7912 - val_accuracy: 0.7665
Epoch 3/10
517/517 [==============================] - 6s 12ms/step - loss: 4.1257 - accuracy: 0.8571 - val_loss: 7.3967 - val_accuracy: 0.7677
Epoch 4/10
517/517 [==============================] - 8s 15ms/step - loss: 3.0934 - accuracy: 0.8980 - val_loss: 8.1721 - val_accuracy: 0.7548
Epoch 5/10
517/517 [==============================] - 7s 13ms/step - loss: 2.2949 - accuracy: 0.9268 - val_loss: 10.1510 - val_accuracy: 0.7381
Epoch 6/10
517/517 [==============================] - 6s 11ms/step - loss: 1.9726 - accuracy: 0.9394 - val_loss: 12.3605 - val_accuracy: 0.7345
Epoch 7/10
517/517 [==============================] - 5s 10ms/step - loss: 1.7782 - accuracy: 0.9458 - val_loss: 13.4252 - val_accuracy: 0

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(np.argmax(model.predict(X_test_array), axis=1), y_test_array))

130/130 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.56      0.51      0.54       500
           1       0.12      0.18      0.15       103
           2       0.14      0.20      0.17       136
           3       0.14      0.29      0.19       199
           4       0.93      0.86      0.89      3194

    accuracy                           0.75      4132
   macro avg       0.38      0.41      0.39      4132
weighted avg       0.80      0.75      0.77      4132



### Теперь с предобученными весами для слоя Embedding

Данные были взяты здесь: http://vectors.nlpl.eu/repository/20/220.zip

In [ ]:
with open('/content/drive/MyDrive/Colab_Notebooks/25_NLP/work_7/data/220/model.txt', 'r') as f:
    data = f.read()

In [ ]:
data = [line.split() for line in data.split('\n')]
data[0][:10]

['249333', '300']

In [ ]:
from tqdm import tqdm

In [ ]:
word2vec = {}
for word in tqdm(data[:-1]):
  word2vec[word[0].split('_')[0]] = np.array(word[1:]).astype(float)
list(word2vec.items())[0]

100%|██████████| 249334/249334 [00:49<00:00, 4989.71it/s]


('249333', array([300.]))

In [ ]:
word2num = {j:i for i,j in enumerate(word2vec.keys(), 1)}
list(word2num.items())[:10]

[('249333', 1),
 ('xxxxxxxx', 2),
 ('год', 3),
 ('xxxxxx', 4),
 ('xxxxxxx', 5),
 ('первый', 6),
 ('время', 7),
 ('человек', 8),
 ('introduction', 9),
 ('один', 10)]

In [ ]:
MAX_WORDS = len(word2num)
MAX_WORDS

205708

In [ ]:
X_train['Content_sequence_word2num'] = X_train['Content_preprocess'].apply(lambda text: [word2num.get(word, 0) for word in text.split()][:MAX_LEN]+[0]*(MAX_LEN-len(text.split())))
X_test['Content_sequence_word2num'] = X_test['Content_preprocess'].apply(lambda text: [word2num.get(word, 0) for word in text.split()][:MAX_LEN]+[0]*(MAX_LEN-len(text.split())))
X_train.head()

,Content,Date,Content_preprocess,Content_sequence,Content_sequence_word2num
2680,Очень удобно!!!,2017-08-10,удобно,"[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[32634, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
844,Не приходит смс код для входа в приложение,2017-08-12,приходить смс код вход приложение,"[57, 45, 42, 18, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[1126, 21771, 621, 1868, 2235, 0, 0, 0, 0, 0, ..."
18714,Пока все удобно и понятно.,2017-07-01,удобно понятно,"[2, 46, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[32634, 28407, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
13842,Удобное приложение. Работает отлично.,2017-07-24,удобный приложение работать отлично,"[4, 1, 3, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2713, 2235, 62, 5154, 0, 0, 0, 0, 0, 0, 0, 0,..."
11974,Очень удобное приложение!!,2017-07-28,удобный приложение,"[4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2713, 2235, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
num2vec = {word2num[i]:j for i,j in word2vec.items()}

In [ ]:
embedding_dim = len(list(word2vec.items())[0][1])
embedding_dim

In [ ]:
zero_vec = [0]*embedding_dim

In [ ]:
num2vec[0] = zero_vec

In [ ]:
embedding_start_weights = np.array([j for i,j in sorted(num2vec.items(), key=lambda x: x[0])])
embedding_start_weights.shape

<ipython-input-41-8b49ebc23827>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  embedding_start_weights = np.array([j for i,j in sorted(num2vec.items(), key=lambda x: x[0])])


(205709,)

In [ ]:
X_train_array = np.array([[j for j in i] for i in X_train['Content_sequence_word2num']])
X_test_array = np.array([[j for j in i] for i in X_test['Content_sequence_word2num']])
X_train_array.shape

(16527, 100)

In [ ]:
embedding_dim = 300
hidden_dim = 512

In [ ]:
model = Sequential([
    Embedding(MAX_WORDS+1, embedding_dim),
    Conv1D(embedding_dim*2, 3),
    GlobalMaxPooling1D(),
    Dense(hidden_dim, activation='relu'),
    Dense(hidden_dim//2, activation='relu'),

    Dense(units=5, activation='sigmoid')
])

In [ ]:
model.layers[0].get_weights()[0].shape

(205709, 300)

In [ ]:
embedding_start_weights.shape

(205709,)

Инициализируем веса:

In [ ]:
model.layers[0].set_weights([embedding_start_weights])

ValueError: ignored

In [ ]:
model.build((None, MAX_LEN, embedding_dim))
model.summary()
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              loss_weights=disbalance,
              optimizer='adam',
              metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 300)         61712700  
                                                                 
 conv1d_1 (Conv1D)           (None, None, 600)         540600    
                                                                 
 global_max_pooling1d_1 (Glo  (None, 600)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_3 (Dense)             (None, 512)               307712    
                                                                 
 dense_4 (Dense)             (None, 256)               131328    
                                                                 
 dense_5 (Dense)             (None, 5)                 1285      
                                                      

In [ ]:
with tf.device("GPU:0"):
  hh = model.fit(X_train_array, y_train_array, validation_data=(X_test_array, y_test_array), epochs=10)

Epoch 1/10
517/517 [==============================] - 41s 72ms/step - loss: 6.9520 - accuracy: 0.7629 - val_loss: 6.9886 - val_accuracy: 0.7662
Epoch 2/10
517/517 [==============================] - 15s 30ms/step - loss: 5.8554 - accuracy: 0.7965 - val_loss: 6.6199 - val_accuracy: 0.7740
Epoch 3/10
517/517 [==============================] - 12s 24ms/step - loss: 5.0341 - accuracy: 0.8222 - val_loss: 7.0128 - val_accuracy: 0.7640
Epoch 4/10
517/517 [==============================] - 13s 25ms/step - loss: 4.2882 - accuracy: 0.8542 - val_loss: 7.7749 - val_accuracy: 0.7505
Epoch 5/10
517/517 [==============================] - 13s 25ms/step - loss: 3.5936 - accuracy: 0.8806 - val_loss: 9.0820 - val_accuracy: 0.7478
Epoch 6/10
517/517 [==============================] - 11s 21ms/step - loss: 3.0167 - accuracy: 0.9044 - val_loss: 10.6143 - val_accuracy: 0.7466
Epoch 7/10
517/517 [==============================] - 11s 22ms/step - loss: 2.6563 - accuracy: 0.9155 - val_loss: 11.8314 - val_accurac

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(np.argmax(model.predict(X_test_array), axis=1), y_test_array))

130/130 [==============================] - 0s 3ms/step
              precision    recall  f1-score   support

           0       0.47      0.58      0.52       372
           1       0.09      0.14      0.11        98
           2       0.15      0.17      0.16       160
           3       0.13      0.30      0.19       180
           4       0.95      0.83      0.89      3322

    accuracy                           0.75      4132
   macro avg       0.36      0.41      0.37      4132
weighted avg       0.82      0.75      0.78      4132



### Теперь попробуем без дообучения, заморозим слой Embedding

In [ ]:
model = Sequential([
    Embedding(MAX_WORDS+1, embedding_dim),
    Conv1D(embedding_dim*2, 3),
    GlobalMaxPooling1D(),
    Dense(hidden_dim, activation='relu'),
    Dense(hidden_dim//2, activation='relu'),

    Dense(units=5, activation='sigmoid')
])

In [ ]:
model.layers[0].set_weights([embedding_start_weights])

ValueError: ignored

In [ ]:
model.layers[0].trainable=False

In [ ]:
model.build((None, MAX_LEN, embedding_dim))
model.summary()
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              loss_weights=disbalance,
              optimizer='adam',
              metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 300)         61712700  
                                                                 
 conv1d_2 (Conv1D)           (None, None, 600)         540600    
                                                                 
 global_max_pooling1d_2 (Glo  (None, 600)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_6 (Dense)             (None, 512)               307712    
                                                                 
 dense_7 (Dense)             (None, 256)               131328    
                                                                 
 dense_8 (Dense)             (None, 5)                 1285      
                                                      

In [ ]:
with tf.device("GPU:0"):
  hh = model.fit(X_train_array, y_train_array, validation_data=(X_test_array, y_test_array), epochs=10)

Epoch 1/10
517/517 [==============================] - 6s 8ms/step - loss: 7.3158 - accuracy: 0.7476 - val_loss: 6.8889 - val_accuracy: 0.7570
Epoch 2/10
517/517 [==============================] - 3s 6ms/step - loss: 6.4386 - accuracy: 0.7774 - val_loss: 6.7127 - val_accuracy: 0.7633
Epoch 3/10
517/517 [==============================] - 3s 6ms/step - loss: 5.7616 - accuracy: 0.8023 - val_loss: 6.9189 - val_accuracy: 0.7674
Epoch 4/10
517/517 [==============================] - 4s 7ms/step - loss: 4.9607 - accuracy: 0.8315 - val_loss: 7.7159 - val_accuracy: 0.7517
Epoch 5/10
517/517 [==============================] - 4s 7ms/step - loss: 4.1965 - accuracy: 0.8586 - val_loss: 9.0690 - val_accuracy: 0.7623
Epoch 6/10
517/517 [==============================] - 3s 6ms/step - loss: 3.5986 - accuracy: 0.8822 - val_loss: 9.6861 - val_accuracy: 0.7430
Epoch 7/10
517/517 [==============================] - 3s 6ms/step - loss: 3.0824 - accuracy: 0.9023 - val_loss: 10.9615 - val_accuracy: 0.7544
Epoch

In [ ]:
print(classification_report(np.argmax(model.predict(X_test_array), axis=1), y_test_array))

130/130 [==============================] - 0s 3ms/step
              precision    recall  f1-score   support

           0       0.62      0.51      0.56       552
           1       0.06      0.15      0.08        60
           2       0.10      0.14      0.12       125
           3       0.14      0.31      0.19       182
           4       0.94      0.86      0.90      3213

    accuracy                           0.75      4132
   macro avg       0.37      0.39      0.37      4132
weighted avg       0.82      0.75      0.78      4132



### Посмотрим как будет обучаться модель со случайными весами Embedding слоя

In [ ]:
model = Sequential([
    Embedding(MAX_WORDS+1, embedding_dim),
    Conv1D(embedding_dim*2, 3),
    GlobalMaxPooling1D(),
    Dense(hidden_dim, activation='relu'),
    Dense(hidden_dim//2, activation='relu'),

    Dense(units=5, activation='sigmoid')
])

In [ ]:
model.layers[0].trainable=False

In [ ]:
model.build((None, MAX_LEN, embedding_dim))
model.summary()
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              loss_weights=disbalance,
              optimizer='adam',
              metrics=['accuracy'])

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 300)         61712700  
                                                                 
 conv1d_3 (Conv1D)           (None, None, 600)         540600    
                                                                 
 global_max_pooling1d_3 (Glo  (None, 600)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_9 (Dense)             (None, 512)               307712    
                                                                 
 dense_10 (Dense)            (None, 256)               131328    
                                                                 
 dense_11 (Dense)            (None, 5)                 1285      
                                                      

In [ ]:
with tf.device("GPU:0"):
  hh = model.fit(X_train_array, y_train_array, validation_data=(X_test_array, y_test_array), epochs=10)

Epoch 1/10
517/517 [==============================] - 5s 7ms/step - loss: 7.2980 - accuracy: 0.7522 - val_loss: 6.8244 - val_accuracy: 0.7628
Epoch 2/10
517/517 [==============================] - 4s 7ms/step - loss: 6.3791 - accuracy: 0.7779 - val_loss: 6.9776 - val_accuracy: 0.7669
Epoch 3/10
517/517 [==============================] - 3s 6ms/step - loss: 5.6843 - accuracy: 0.8024 - val_loss: 7.0005 - val_accuracy: 0.7587
Epoch 4/10
517/517 [==============================] - 3s 6ms/step - loss: 4.8692 - accuracy: 0.8323 - val_loss: 7.5687 - val_accuracy: 0.7544
Epoch 5/10
517/517 [==============================] - 4s 7ms/step - loss: 4.0840 - accuracy: 0.8636 - val_loss: 9.3308 - val_accuracy: 0.7531
Epoch 6/10
517/517 [==============================] - 4s 7ms/step - loss: 3.5430 - accuracy: 0.8841 - val_loss: 10.0920 - val_accuracy: 0.7348
Epoch 7/10
517/517 [==============================] - 3s 6ms/step - loss: 3.1262 - accuracy: 0.9003 - val_loss: 11.1243 - val_accuracy: 0.7580
Epoc

In [ ]:
print(classification_report(np.argmax(model.predict(X_test_array), axis=1), y_test_array))

130/130 [==============================] - 1s 5ms/step
              precision    recall  f1-score   support

           0       0.65      0.48      0.55       620
           1       0.05      0.23      0.08        35
           2       0.12      0.16      0.14       135
           3       0.22      0.26      0.24       336
           4       0.90      0.88      0.89      3006

    accuracy                           0.74      4132
   macro avg       0.39      0.40      0.38      4132
weighted avg       0.78      0.74      0.76      4132



Метрики остались такими же. Даже со случайными векторами сетка может обучаться до 0.92 accuracy на train.

### Вывод:

Все результаты оказались приблизительно одинаково плохими, возможно стоит обратить внимание на сами данные и как делается их предобработка. Так же можно попробывать dropout и batchnorm чтоби снизить явное переобучение. Возможно стоит поискать word2vec веса получше. 

Стоит отметить, что метрика на train при обучении Embedding без предобученных весов оказалась выше. Так как модель начала быстро затачиваться под конкретные слова и набирать большую метрику.

Так как в последнем эксперименте сетка смогла обучиться даже на случайных векторах (хотя метрика на train меньше, чем когда мы обучали слой Embedding), то очевидно, что проблема в переобучении. Поэтому есть веский повод попробывать заменить некоторые слова на сущности/тэги, которые так кстати давались вместе с word2vec данными. Тогда переобучение должно будет уменьшиться.